In [1]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
import os

import yaml
import hashlib

In [2]:
N = sp.symbols("N", integer=True)
t = sp.symbols('t')
_N = 2

γ = sp.symbols('γ')
E0, L, w1, σc = sp.symbols('E0 L w1 σc', positive = True)

u = sp.symbols([f"u{i}" for i in range(0, _N)])
e = sp.symbols([f"e{i}" for i in range(1, _N+1)])
α = sp.symbols([f"α{i}" for i in range(1, _N+1)])
_α = sp.symbols("α")



β = sp.symbols('β')



In [3]:
state = {"u": u, "α": α, "e": e}
_matpar = {N: _N, γ: 2, E0: 1, w1: 1, L: 1}
matpar = {"N": N, "γ": γ, "E0": E0, "w1": w1, "L": L}



In [4]:
_a = sp.Function('a')
_w = sp.Function('w')
_s = sp.Function('s')


In [5]:
_s = 1/_a(_α)

In [6]:
_s

1/a(α)

In [7]:
_ap = sp.diff(_a(_α), _α)

In [8]:
class SymbolicDiscreteDamage:
    def __init__(self, state, matpar, 
                 name = "discrete generalised damage",
                 slug = "general damage"):
        self.state = state
        self.matpar = matpar
        self.str = name
        self.slug = slug

    def _s(self, αi):
        s = sp.Function('s')
        # s = 1/self._a(αi)
        return s(αi)

    def _a(self, αi):
        γ = self.matpar["γ"]
        a = sp.Function('a')
        return a(αi)
    
    def energy(self, state):
        """Total energy"""
        return self._elastic(state) + self._damage(state) - self.work(state)

    def work(self, state):
        return 0

    def _elastic(self, state):
        _e = state["e"]
        _α = state["α"]
        # _w1 = matpar["w1"]
        _E0 = self.matpar["E0"]
        _N = self.matpar["N"]
    
        return sum([1./2. * E0 * N * self._a(α[i]) * e[i]**2 for i in range(len(α))])

    def _w(self, αi):
        w = sp.Function('w')

        return w(αi)

    def _damage(self, state):
        _α = state["α"]
        _w1 = self.matpar["w1"]

        _N = self.matpar["N"]

        return sum([_w1 / _N * self._w(_α[i]) for i in range(len(_α))]) 
        # return _w1 * self._w(_α)


    # Explicit expressions
    def w(self, αi):
        return αi

    def a(self, αi):
        γ = self.matpar["γ"]

        return (1 - self._w(αi)) / ((γ-1) * self._w(αi) + 1)


    def __str__(self):
     return self.str


In [9]:
atls = SymbolicDiscreteDamage(state, matpar)


In [10]:
atls._s

<bound method SymbolicDiscreteDamage._s of <__main__.SymbolicDiscreteDamage object at 0x119bdd8e0>>

In [11]:
atls._s(α[0])

s(α1)

In [12]:
atls.energy(state)

0.5*E0*N*e1**2*a(α1) + 0.5*E0*N*e2**2*a(α2) + w1*w(α1)/N + w1*w(α2)/N

In [171]:
class ModelAnalysis:
    """Helper class to analyse stability properties of a model"""

    def __init__(self, model):
        self.model = model
        self.state = model.state
        _state = self.state
        self.matpar = model.matpar

        _β = sp.symbols('β')
        _e = sp.symbols('e')
        self._β = _β
        self._e = _e
        
        self._s = model._s(_β)
        self._sp = sp.diff(self._s, _β, 1)
        self._spp = sp.diff(self._s, _β, 2)

        self._a = model._a(_β)
        self._ap = sp.diff(self._a, _β, 1)
        self._app = sp.diff(self._a, _β, 2)

        self._w = model._w(_β)
        self._wp = sp.diff(self._w, _β)
        self._wpp = sp.diff(self._w, _β, 2)

        # self.criterion()


    def criterion(self):
        """Damage onset"""
        model, state = self.model, self.state
        x = sp.symbols('x')

        β = self._β
        e = self._e

        _u0 = t*x/L
        _e0 = t/L
        _alpha0 = 0

        _homogeneous_damage = {a: β for a in state["α"]}
        _homogeneous_strain = {a: e for a in state["e"]}
        _sound = {a: 0 for a in state["α"]}

        _crit = sp.diff(
            model.energy(state)                 \
                .subs(_homogeneous_damage)      \
                .subs(_homogeneous_strain), β)  \
                .subs({e: _e0, β: _alpha0})    

        self.crit = _crit

        return _crit

    def critical_load(self, matpar = {}):
        """Critical load"""

        criterion = self.crit           \
            .replace(_a, self.model.a)  \
            .replace(_w, self.model.w)

        tc = sp.solve(criterion, t)
        
        if matpar:
            tc = [_tc.subs(_matpar) for _tc in tc if _tc.subs(_matpar).is_positive]
        
        return tc, criterion


    def _equilirbrium_e(self):
        _S = sum([self.model._s(αi) for αi in state["α"]])

        _e = [(t/L * self.model._s(αi) / _S).simplify() \
                    for αi in state["α"]]

        return _e
    
    def energy_fc_alpha(self):
        _ei = self._equilirbrium_e()
        equilibrium_subs = {v[0]: v[1] for v in zip(state["e"], _ei)}
        
        return self.model.energy(state).subs(equilibrium_subs)


    def _state_split(self, m, state, matpar):
        """Split the state into homogeneous and bifurcated parts"""

        assert m < matpar[N]

        _bif_α = [state["α"][k+1] for k in range(m)]
        _bif_e = [state["e"][k+1] for k in range(m)]

        _homo_α = set(state["α"]) - set(_bif_α)
        _homo_e = set(state["e"]) - set(_bif_e)

        return _bif_α, _homo_α, _bif_e, _homo_e

    def _state_homog_substitutions(self, state, matpar):
        """Substitutions for the homogeneous state"""
        αt = sp.symbols('α_t')
        et = sp.symbols('e_t')

        _subs_αh = {a: αt for a in state["α"]}
        _subs_eh = {e: et for e in state["e"]}

        return _subs_αh, _subs_eh

    def homogeneous_α(self, matpar = {}):
        """Solves the (homogeneous) evolution law for damage"""
        αt = sp.symbols('α_t')
        
        if matpar:
            return [sol.subs(matpar) for sol in \
                    list(map(sp.simplify, sp.solve(self.evo_α(matpar), αt)))]
        else:
            return list(map(sp.simplify, sp.solve(self.evo_α(), αt)))

    def evo_α(self, matpar = {}):
        """Evolution law for damage"""
        αt = sp.symbols('α_t')
        subs_homog_α, _ =  self._state_homog_substitutions(state, matpar)

        return sp.diff(self.energy_fc_alpha().subs(subs_homog_α), αt)                   \
            .replace(_a, self.model.a)                  \
            .replace(_w, self.model.w)                  \
            .doit().simplify()


In [142]:
model = ModelAnalysis(atls)
model.criterion()

1.0*E0*N*t**2*Subs(Derivative(a(β), β), β, 0)/L**2 + 2*w1*Subs(Derivative(w(β), β), β, 0)/N

In [143]:
tc, criterion = model.critical_load()

In [51]:
tc

[-1.4142135623731*L*sqrt(w1)*sqrt(1/γ)/(sqrt(E0)*N),
 1.4142135623731*L*sqrt(w1)*sqrt(1/γ)/(sqrt(E0)*N)]

In [17]:
tc, criterion = model.critical_load(_matpar)

In [18]:
tc, criterion

([0.353553390593274*sqrt(2)],
 1.0*E0*N*t**2*Subs(Derivative((1 - β)/(β*(γ - 1) + 1), β), β, 0)/L**2 + 2*w1*Subs(Derivative(β, β), β, 0)/N)

In [19]:
[_tc.subs(_matpar) for _tc in tc if _tc.subs(_matpar).is_positive]

[0.353553390593274*sqrt(2)]

In [20]:
ModelAnalysis(atls).criterion().doit()


1.0*E0*N*t**2*Subs(Derivative(a(β), β), β, 0)/L**2 + 2*w1*Subs(Derivative(w(β), β), β, 0)/N

In [21]:
_asd = ModelAnalysis(atls).criterion().replace(_a, lambda x: ((1-x))**(2) )    \
                                .replace(_w, lambda x: x).doit()

In [22]:
_homogeneous = {a: β for a in state["α"]}
_sound = {a: 0 for a in state["α"]}



In [23]:
{a: 0 for a in state["α"]}

{α1: 0, α2: 0}

In [24]:
sp.diff(atls.energy(state).subs(_homogeneous), β).simplify()

(0.5*E0*N**2*(e1**2 + e2**2)*Derivative(a(β), β) + 2*w1*Derivative(w(β), β))/N

## Model Analysis

- (constitutive assumptions)
- equilibrium
- homogeneous solution
- homogeneous energy
- bifurcations
- stability

In [172]:
N = sp.symbols("N", integer=True)
t = sp.symbols('t')
_N = 2

γ = sp.symbols('γ')
E0, L, w1, σc = sp.symbols('E0 L w1 σc', positive = True)

u = sp.symbols([f"u{i}" for i in range(0, _N)])
e = sp.symbols([f"e{i}" for i in range(1, _N+1)])
α = sp.symbols([f"α{i}" for i in range(1, _N+1)])
_α = sp.symbols("α")



β = sp.symbols('β')
αt = sp.symbols('α_t')


In [173]:
atls = SymbolicDiscreteDamage(state, matpar)
model = ModelAnalysis(atls)
model.criterion()

1.0*E0*N*t**2*Subs(Derivative(a(β), β), β, 0)/L**2 + 2*w1*Subs(Derivative(w(β), β), β, 0)/N

In [174]:
atls.energy(state)

0.5*E0*N*e1**2*a(α1) + 0.5*E0*N*e2**2*a(α2) + w1*w(α1)/N + w1*w(α2)/N

In [63]:
_en = atls._elastic(state) + atls._damage(state)
_en

0.5*E0*N*e1**2*a(α1) + 0.5*E0*N*e2**2*a(α2) + w1*w(α1)/N + w1*w(α2)/N

In [56]:
state

{'u': [u0, u1], 'α': [α1, α2], 'e': [e1, e2]}

In [30]:
model.energy_fc_alpha()

0.5*E0*N*t**2*a(α1)*s(α1)**2/(L**2*(s(α1) + s(α2))**2) + 0.5*E0*N*t**2*a(α2)*s(α2)**2/(L**2*(s(α1) + s(α2))**2) + w1*w(α1)/N + w1*w(α2)/N

### homogeneous solution

In [65]:
bif_α, homo_α, bif_e, homo_e = model._state_split(0, state, _matpar)

In [36]:
bif_α, homo_α

([], {α1, α2})

In [67]:
subs_homog_α, subs_homog_e =  model._state_homog_substitutions(state, _matpar)

In [69]:
model.energy_fc_alpha().subs(subs_homog_α)

0.25*E0*N*t**2*a(α_t)/L**2 + 2*w1*w(α_t)/N

In [90]:
model.homogeneous_α(_matpar).replace(_a, model.model.a)  \
            .replace(_w, model.model.w).doit()

-1.0*E0*N*t**2*γ/L**2 + 2*w1/N

In [76]:
sp.solve([a.subs(_matpar) for a in sp.solve(model.energy_fc_alpha().subs(subs_homog_α).replace(_a, model.model.a)  \
            .replace(_w, model.model.w), αt)][0], t)

[-99029061.2647496, 99029061.2647496]

In [147]:
model.evo_α()

-0.25*E0*N*t**2*γ/(L**2*(α_t*γ - α_t + 1)**2) + 2*w1/N

In [148]:
model.evo_α(_matpar)

-0.25*E0*N*t**2*γ/(L**2*(α_t*γ - α_t + 1)**2) + 2*w1/N

In [175]:
model.homogeneous_α()

[(-0.353553390593274*sqrt(E0)*N*t*sqrt(γ)*(γ - 1.0)**2 - L*sqrt(w1)*(γ**2 - 2.0*γ + 1.0))/(L*sqrt(w1)*(γ - 1.0)*(γ**2 - 2.0*γ + 1.0)),
 (0.353553390593274*sqrt(E0)*N*t*sqrt(γ)*(γ - 1.0)**2 - L*sqrt(w1)*(γ**2 - 2.0*γ + 1.0))/(L*sqrt(w1)*(γ - 1.0)*(γ**2 - 2.0*γ + 1.0))]

In [176]:
model.homogeneous_α(_matpar)

[-0.707106781186548*sqrt(2)*t - 1.0, 0.707106781186548*sqrt(2)*t - 1.0]